In [1]:
// mkdir data
// wget -nc http://cedric.cnam.fr/vertigo/Cours/RCP216/docs/a7.zip

Intitializing Scala interpreter ...

Spark Web UI available at http://jeslava-pc:4040
SparkContext available as 'sc' (version = 2.4.4, master = local[*], app id = local-1585679542909)
SparkSession available as 'spark'


In [2]:
// Lire les données dans le format approprié
val donnees = spark.read.format("libsvm").load("file:///home/jeslava/msata/cnam_RCP216/data_mining/TP7_SVM_Linear/data/a7")
donnees.show(2, false)  // visualiser les 2 premières lignes du DataFrame

+-----+----------------------------------------------------------------------------------------------------------+
|label|features                                                                                                  |
+-----+----------------------------------------------------------------------------------------------------------+
|0.0  |(123,[2,10,13,18,38,41,54,63,66,72,74,75,79,82],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|0.0  |(123,[2,5,16,21,35,40,52,63,66,72,73,75,79,82],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]) |
+-----+----------------------------------------------------------------------------------------------------------+
only showing top 2 rows



donnees: org.apache.spark.sql.DataFrame = [label: double, features: vector]


In [3]:
// Partitionner les données en apprentissage (50%) et test (50%)
val partitions = donnees.randomSplit(Array(0.5, 0.5))
val apprentissage = partitions(0).cache()  // conservé en mémoire
val test = partitions(1)

partitions: Array[org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]] = Array([label: double, features: vector], [label: double, features: vector])
apprentissage: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: double, features: vector]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: double, features: vector]


In [4]:
// Construction de pipeline
import org.apache.spark.ml.classification.LinearSVC
import org.apache.spark.ml.{Pipeline, PipelineModel}

// Définition de l'estimateur SVC linéaire
val linSvc = new LinearSVC().setMaxIter(10)
                            .setFeaturesCol("features")
                            .setLabelCol("label")

// Définition du pipeline (réduit ici au SVM linéaire)
val pipeline = new Pipeline().setStages(Array(linSvc))

import org.apache.spark.ml.classification.LinearSVC
import org.apache.spark.ml.{Pipeline, PipelineModel}
linSvc: org.apache.spark.ml.classification.LinearSVC = linearsvc_2628fa455300
pipeline: org.apache.spark.ml.Pipeline = pipeline_d146b9079f16


In [5]:
import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

// Construction de la grille de (hyper)paramètres utilisée pour grid search
// Une composante est ajoutée avec .addGrid() pour chaque (hyper)paramètre à explorer
val paramGrid = new ParamGridBuilder()
                           .addGrid(linSvc.regParam,
                                    Array(0.02, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5))
                           .build()

// Définition de l'instance de CrossValidator : à quel estimateur l'appliquer,
//  avec quels (hyper)paramètres, combien de folds, comment évaluer les résultats
val cv = new CrossValidator().setEstimator(pipeline)
                             .setEstimatorParamMaps(paramGrid)
                             .setNumFolds(5)
                             .setEvaluator(new BinaryClassificationEvaluator())

// Construction et évaluation par validation croisée des modèles correspondant
//   à toutes les combinaisons de valeurs de (hyper)paramètres de paramGrid
val cvSVCmodel = cv.fit(apprentissage)

import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	linearsvc_2628fa455300-regParam: 0.02
}, {
	linearsvc_2628fa455300-regParam: 0.05
}, {
	linearsvc_2628fa455300-regParam: 0.1
}, {
	linearsvc_2628fa455300-regParam: 0.2
}, {
	linearsvc_2628fa455300-regParam: 0.3
}, {
	linearsvc_2628fa455300-regParam: 0.4
}, {
	linearsvc_2628fa455300-regParam: 0.5
})
cv: org.apache.spark.ml.tuning.CrossValidator = cv_bd970a1cacd2
cvSVCmodel: org.apache.spark.ml.tuning.CrossValidatorModel = cv_bd970a1cacd2


In [6]:
// Afficher les valeurs AUC obtenues pour les combinaisons de la grille
cvSVCmodel.avgMetrics

res2: Array[Double] = Array(0.8937221590149094, 0.8919098051750863, 0.893076203632652, 0.8923139510919373, 0.8916698228699008, 0.8916453922738434, 0.8910546328127511)


In [7]:
// Afficher les meilleures valeurs pour les hyperparamètres
cvSVCmodel.getEstimatorParamMaps.zip(cvSVCmodel.avgMetrics).maxBy(_._2)._1

res3: org.apache.spark.ml.param.ParamMap =
{
	linearsvc_2628fa455300-regParam: 0.02
}


In [9]:
// Enregistrement du meilleur modèle
cvSVCmodel.write.save("file:///home/jeslava/msata/cnam_RCP216/data_mining/TP7_SVM_Linear/cvSVCmodel")

In [10]:
//Évaluation du modèle obtenu avec les valeurs optimales des hyper-paramètres
// Calculer les prédictions sur les données d'apprentissage
val resApp = cvSVCmodel.transform(apprentissage)

// Calculer les prédictions sur les données de test
val resultats = cvSVCmodel.transform(test)

// Afficher le schéma du DataFrame resultats
resultats.printSchema()

// Afficher 10 lignes complètes de résultats (sans la colonne features)
resultats.select("label", "rawPrediction", "prediction").show(10, false)

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- prediction: double (nullable = false)

+-----+------------------------------------------+----------+
|label|rawPrediction                             |prediction|
+-----+------------------------------------------+----------+
|0.0  |[0.7921919119369204,-0.7921919119369204]  |0.0       |
|0.0  |[-0.17729742321150285,0.17729742321150285]|1.0       |
|0.0  |[0.38845192144805984,-0.38845192144805984]|0.0       |
|0.0  |[0.5091415603678989,-0.5091415603678989]  |0.0       |
|0.0  |[1.2869319403234856,-1.2869319403234856]  |0.0       |
|0.0  |[1.1878277439932219,-1.1878277439932219]  |0.0       |
|0.0  |[1.784685660686986,-1.784685660686986]    |0.0       |
|0.0  |[1.784685660686986,-1.784685660686986]    |0.0       |
|0.0  |[0.8810793402457736,-0.8810793402457736]  |0.0       |
|0.0  |[0.8810793402457736,-0.8810793402457736]  |0.0       |
+-----+----------------

resApp: org.apache.spark.sql.DataFrame = [label: double, features: vector ... 2 more fields]
resultats: org.apache.spark.sql.DataFrame = [label: double, features: vector ... 2 more fields]


In [11]:
// Calculer AUC sur données d'apprentissage
println(cvSVCmodel.getEvaluator.evaluate(resApp))

// Calculer AUC sur données de test
println(cvSVCmodel.getEvaluator.evaluate(resultats))

0.8949034137954631
0.8873289977592711
